In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from custom_py_modules import *
from scipy.optimize import differential_evolution
import json

In [71]:
# Load three years of data

# 5 years in dataframe
df_load = pd.concat([pd.read_csv('./test_simulink/E-load-2013.csv'), 
                     pd.read_csv('./test_simulink/E-load-2014.csv'), 
                     pd.read_csv('./test_simulink/E-load-2015.csv'),
                     pd.read_csv('./test_simulink/E-load-2017.csv'),
                     pd.read_csv('./test_simulink/E-load-2018.csv')
                     ])
df_weather = pd.concat([pd.read_csv('./test_simulink/E-2013.csv'), 
                        pd.read_csv('./test_simulink/E-2014.csv'), 
                        pd.read_csv('./test_simulink/E-2015.csv'),
                        pd.read_csv('./test_simulink/E-2017.csv'),
                        pd.read_csv('./test_simulink/E-2018.csv')
                        ])

# Combine irradiance
df_weather['irradiance_total'] = df_weather['irradiance_direct'] + df_weather['irradiance_diffuse']

# Change the time column names
df_weather['timestamp'] = df_weather['local_time_no_dst']
df_weather = df_weather.drop(columns=['local_time_no_dst'])

df_load['timestamp'] = df_load['DD/MM/YYYY HH:MM']
df_load = df_load.drop(columns=['DD/MM/YYYY HH:MM'])

# Convert to datetime
df_load['timestamp'] = pd.to_datetime(df_load['timestamp'], format='%d/%m/%Y %H:%M')
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'], format='%Y-%m-%d %H:%M:%S')
# Set timestamp as index
df_load = df_load.set_index('timestamp')
df_weather = df_weather.set_index('timestamp')

# Add kW column
df_load['Load (kW)'] = df_load['Load (Watt)'] / 1000
df_load_hourly = df_load.resample('H').mean()

df_load_hourly_2013 = df_load_hourly.loc['2013-01-01':'2013-12-31']
df_load_hourly_2014 = df_load_hourly.loc['2014-01-01':'2014-12-31']
df_load_hourly_2015 = df_load_hourly.loc['2015-01-01':'2015-12-31']
df_load_hourly_2017 = df_load_hourly.loc['2017-01-01':'2017-12-31']
df_load_hourly_2018 = df_load_hourly.loc['2018-01-01':'2018-12-31']

df_weather_hourly_2013 = df_weather.loc['2013-01-01':'2013-12-31']
df_weather_hourly_2014 = df_weather.loc['2014-01-01':'2014-12-31']
df_weather_hourly_2015 = df_weather.loc['2015-01-01':'2015-12-31']
df_weather_hourly_2017 = df_weather.loc['2017-01-01':'2017-12-31']
df_weather_hourly_2018 = df_weather.loc['2018-01-01':'2018-12-31']

In [72]:
# Define wind turbines
wf_builder = WindFarmBuilder()
gazelle_turbine = WindTurbine(blade_radius=5, # [m]
                              performance_coefficient=0.4, # [-]
                              air_density=1.225, # [kg/m^3]
                              nominal_power=20, # [kW] 
                              nominal_wind_speed=13, # [m/s]
                              cut_out_speed=20, # [m/s] 
                              cut_in_speed=4, # [m/s] 
                              price=60000, # EUR from alibaba
                              lifetime=22.5, # years 
                              height=18) # [m]

aircon_hawt_turbine = WindTurbine(blade_radius=3.5, # [m]
                              performance_coefficient=0.35, # [-]
                              air_density=1.225, # [kg/m^3]
                              nominal_power=10, # [kW] 
                              nominal_wind_speed=11, # [m/s]
                              cut_out_speed=32, # [m/s] 
                              cut_in_speed=2.5, # [m/s] 
                              price=35000, # EUR 
                              lifetime=25, # years 
                              height=18) # [m]

travere_turbine = WindTurbine(blade_radius=1.6, # [m]
                              performance_coefficient=0.35, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=1.6, # [kW]
                                nominal_wind_speed=10, # [m/s]
                                cut_out_speed=60, # [m/s]
                                cut_in_speed=2.5 , # [m/s]
                                price=7000, # EUR
                                lifetime=25, # years
                                height=12) # [m]

In [73]:
# In this cell, define the sample energy system

# Wind farm
sample_wind_farm = wf_builder.build_wind_farm([gazelle_turbine, aircon_hawt_turbine, travere_turbine], [7, 0, 0])

# Solar farm
sample_solar_farm = SolarFarm(area=0, # [m^2] 
                                pricepersqm= 325 # in E
                                )

# MicroGrid
sample_microgrid = MicroGrid(windfarm = sample_wind_farm, 
                                solarfarm= sample_solar_farm,
                                storage_capacity_kwh= 13.5*80, # [kWh]
                                storage_efficiency=0.8, # [-] 
                                storage_price_per_kw= 1000 # [EUR]
                                )

sample_net_power = sample_microgrid.get_net_power(
    df_hourlyload_1year=df_load_hourly_2017,
    df_weather_hourly_1year=df_weather_hourly_2017,
    plot=False
)
# Get sum of net power storage over the year
print("Net power total balance: ", sample_net_power['Net Power Storage (kW)'].sum())
# Get som of net power over the year
print("Net power total balance: ", sample_net_power['Net Power (kW)'].sum())

Net power total balance:  350745.16810706473
Net power total balance:  361631.33534696925


In [74]:
imbalance_penalty = 10
sample_net_power_storage = sample_net_power['Net Power Storage (kW)']
sample_net_power_storage.abs().sum() * imbalance_penalty
sample_net_power_storage.abs().resample("M").sum().rolling(window=3).max()
sample_net_power_storage.abs().resample("M").sum().rolling(window=3).mean().max() * imbalance_penalty * 12

print("YEARLY: ", sample_net_power_storage.abs().sum() * imbalance_penalty)
print("AVG SEASONAL WORST: ", sample_net_power_storage.abs().resample("M").sum().rolling(window=3).mean().max() * imbalance_penalty * 12)
print("MONTHLY WORST: ", sample_net_power_storage.abs().resample("M").sum().max() * imbalance_penalty * 12)

YEARLY:  3662288.181975572
AVG SEASONAL WORST:  4361115.047801797
MONTHLY WORST:  5044681.492448775


In [75]:
def get_micro_grid_cost_function_net_zero(solar_price_psqm, 
                                    storage_price_pkw, 
                                    generator_price_per_rated_kw, 
                                    diesel_price, # this kinda serves as penalty for unreliability
                                    gen_fuel_consumption, 
                                    hi_power_turbine_price, 
                                    mid_power_turbine_price, 
                                    low_power_turbine_price, 
                                    list_of_load_dfs,
                                    list_of_weather_dfs,
                                    wf_builder: WindFarmBuilder,
                                    power_inbalance_penalty = 10 # EUR per kW  
                                    ):

    # Define wind 
    gazelle_turbine = WindTurbine(blade_radius=5, # [m]
                                performance_coefficient=0.4, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=20, # [kW] 
                                nominal_wind_speed=13, # [m/s]
                                cut_out_speed=20, # [m/s] 
                                cut_in_speed=4, # [m/s] 
                                price=hi_power_turbine_price, # EUR from alibaba
                                lifetime=22.5, # years 
                                height=18) # [m]

    aircon_hawt_turbine = WindTurbine(blade_radius=3.5, # [m]
                                performance_coefficient=0.35, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=10, # [kW] 
                                nominal_wind_speed=11, # [m/s]
                                cut_out_speed=32, # [m/s] 
                                cut_in_speed=2.5, # [m/s] 
                                price=mid_power_turbine_price, # EUR 
                                lifetime=25, # years 
                                height=18) # [m]

    travere_turbine = WindTurbine(blade_radius=1.6, # [m]
                                performance_coefficient=0.35, # [-]
                                    air_density=1.225, # [kg/m^3]
                                    nominal_power=1.6, # [kW]
                                    nominal_wind_speed=10, # [m/s]
                                    cut_out_speed=60, # [m/s]
                                    cut_in_speed=2.5 , # [m/s]
                                    price=low_power_turbine_price, # EUR
                                    lifetime=25, # years
                                    height=12) # [m]

    def cost_function(x):
        """
        x[0] is number of hi power turbines
        x[1] is number of mid power turbines
        x[2] is number of low power turbines
        x[3] is area of solar panels sqm
        x[4] is storage capacity in kWh
        """

        num_hi_power_turbines = int(round(x[0]))
        num_mid_power_turbines = int(round(x[1]))
        num_low_power_turbines = int(round(x[2]))
        # Build MicroGrid
        sample_wind_farm = wf_builder.build_wind_farm([gazelle_turbine, aircon_hawt_turbine, travere_turbine], [num_hi_power_turbines, num_mid_power_turbines, num_low_power_turbines])
        sample_solar_farm = SolarFarm(area=x[3], # [m^2] 
                                        pricepersqm= solar_price_psqm # in E
                                        )
        sample_microgrid = MicroGrid(windfarm = sample_wind_farm,
                                        solarfarm= sample_solar_farm,
                                        storage_capacity_kwh= x[4], # [kWh]
                                        storage_efficiency=0.8, # [-] 
                                        storage_price_per_kw= storage_price_pkw # [EUR]
                                        )
        system_cost = sample_microgrid.get_total_price()
        seasonal_net_power_costs = []
        annual_net_power_costs = []
        for load_df, weather_df in zip(list_of_load_dfs, list_of_weather_dfs):
            # Get net power for 1 year
            net_power = sample_microgrid.get_net_power(df_hourlyload_1year=load_df,
                                                        df_weather_hourly_1year=weather_df,
                                                        plot=False)
            net_power_storage = net_power['Net Power Storage (kW)']
            annual_net_power_costs.append(net_power_storage.abs().sum() * power_inbalance_penalty)
            # Get the average imbalance of 3 months, and then get the worst of the 3 month periods in the year
            seasonal_net_power_costs.append(net_power_storage.abs().resample("M").sum().rolling(window=3).mean().max() * power_inbalance_penalty * 12)

        # Total cost will be weighted in three types: 40% for cost of the system, 20% for cost of seasonal peak inbalance, 40% for annual inbalance
        total_cost = 0.4 * system_cost + 0.2 * np.mean(seasonal_net_power_costs) + 0.4 * np.mean(annual_net_power_costs)
        
        return total_cost
    return cost_function

In [76]:
default_solar_price_psqm = 325
default_storage_price_pkw = 1000
default_generator_price_per_rated_kw = 375
default_diesel_price = 1.8
default_gen_fuel_consumption = 0.37
default_hi_power_turbine_price = 60000
default_mid_power_turbine_price = 35000
default_low_power_turbine_price = 7000
wf_builder = WindFarmBuilder()

realistic_scenario = get_micro_grid_cost_function_net_zero(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                power_inbalance_penalty=10
                                                )

expensive_diesel_scenario = get_micro_grid_cost_function_net_zero(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=2.3,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                )

cheap_solar_scenario = get_micro_grid_cost_function_net_zero(solar_price_psqm=200,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                )

cheap_wind_scenario = get_micro_grid_cost_function_net_zero(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=48000,
                                                mid_power_turbine_price=35000,
                                                low_power_turbine_price=5600,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                )

cheap_batteries_scenario = get_micro_grid_cost_function_net_zero(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=800,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                )

super_expensive_diesel_scenario = get_micro_grid_cost_function_net_zero(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=3,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                )

super_cheap_storage_scenario = get_micro_grid_cost_function_net_zero(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=666,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                )


# Example usage
total_cost = realistic_scenario([7, 0, 0, 250, 0])
print(total_cost)
# Set bounds for the variables
# x[0] is number of hi power turbines
# x[1] is number of mid power turbines
# x[2] is number of low power turbines
# x[3] is area of solar panels sqm
# x[4] is storage capacity in kWh

# Define bounds for the variables
bounds = [(0, 50),  # Number of high power turbines
          (0, 100),  # Number of mid power turbines
          (0, 100),  # Number of low power turbines
          (0, 25000),  # Area of solar panels in sqm
          (0, 10000)]  # Storage capacity in kWh

3293699.1190522923


In [77]:
def get_optimized_results(cost_function, bounds, name: str, maxiter = 50, export = True):
    # Check if file already exists, if it does return the results from it
    try:
        with open(f'./results/grid_connected/{name}_{maxiter}.json') as json_file:
            result_dict = json.load(json_file)
            print(f"{name} already exists.")
            return result_dict
    except:
        pass
    # Run differential evolution
    # x[0] is number of hi power turbines
    # x[1] is number of mid power turbines
    # x[2] is number of low power turbines
    # x[3] is area of solar panels sqm
    # x[4] is storage capacity in kWh
    result = differential_evolution(cost_function, bounds, maxiter=maxiter, disp=True)
    # Create dictionary of results
    result_dict = {'Number of hi power turbines': int(round(result.x[0])),
                   'Number of mid power turbines': int(round(result.x[1])),
                   'Number of low power turbines': int(round(result.x[2])),
                   'Area of solar panels (sqm)': result.x[3],
                   'Storage capacity (kWh)': result.x[4],
                   'Total cost (EUR)': result.fun}
    
    if export:
        with open(f'./results/grid_connected/{name}_{maxiter}.json', 'w') as fp:
            json.dump(result_dict, fp)
    return result_dict

In [78]:
get_optimized_results(realistic_scenario, bounds, 'realistic_result', maxiter=5, export=True)
get_optimized_results(expensive_diesel_scenario, bounds, 'expensive_diesel_result', maxiter=5, export=True)
get_optimized_results(cheap_solar_scenario, bounds, 'cheap_solar_result', maxiter=5, export=True)
get_optimized_results(cheap_wind_scenario, bounds, 'cheap_wind_result', maxiter=5, export=True)
get_optimized_results(cheap_batteries_scenario, bounds, 'cheap_batteries_result', maxiter=5, export=True)
get_optimized_results(super_expensive_diesel_scenario, bounds, 'super_expensive_diesel_result', maxiter=5, export=True)
get_optimized_results(super_cheap_storage_scenario, bounds, 'super_cheap_storage_result', maxiter=5, export=True)

realistic_result = get_optimized_results(realistic_scenario, bounds, 'realistic_result', maxiter=50, export=True)
expensive_diesel_result = get_optimized_results(expensive_diesel_scenario, bounds, 'expensive_diesel_result', maxiter=50, export=True)
cheap_solar_result = get_optimized_results(cheap_solar_scenario, bounds, 'cheap_solar_result', maxiter=50, export=True)
cheap_wind_result = get_optimized_results(cheap_wind_scenario, bounds, 'cheap_wind_result', maxiter=50, export=True)
cheap_batteries_result = get_optimized_results(cheap_batteries_scenario, bounds, 'cheap_batteries_result', maxiter=50, export=True)
super_expensive_diesel_result = get_optimized_results(super_expensive_diesel_scenario, bounds, 'super_expensive_diesel_result', maxiter=50, export=True)
super_cheap_storage_result = get_optimized_results(super_cheap_storage_scenario, bounds, 'super_cheap_storage_result', maxiter=50, export=True)

realistic_result already exists.
expensive_diesel_result already exists.
cheap_solar_result already exists.
cheap_wind_result already exists.
cheap_batteries_result already exists.
super_expensive_diesel_result already exists.
super_cheap_storage_result already exists.
differential_evolution step 1: f(x)= 1.52712e+07
differential_evolution step 2: f(x)= 9.57032e+06
differential_evolution step 3: f(x)= 9.57032e+06
differential_evolution step 4: f(x)= 3.7961e+06
differential_evolution step 5: f(x)= 1.66253e+06
differential_evolution step 6: f(x)= 930872
differential_evolution step 7: f(x)= 930872
differential_evolution step 8: f(x)= 930872
differential_evolution step 9: f(x)= 930872
differential_evolution step 10: f(x)= 930872
differential_evolution step 11: f(x)= 930872
differential_evolution step 12: f(x)= 930872
differential_evolution step 13: f(x)= 930872
differential_evolution step 14: f(x)= 930872
differential_evolution step 15: f(x)= 930872
differential_evolution step 16: f(x)= 93